In [ ]:
import rospy
import pickle
import moveit_commander 
from sensor_msgs.msg import JointState
from moveit_msgs.msg import DisplayTrajectory, RobotTrajectory
import random


In [ ]:
import moveit_msgs.msg
rospy.init_node('notebook')
state_pub = rospy.Publisher("notebook_state", moveit_msgs.msg.DisplayRobotState, queue_size=1)
traj_pub = rospy.Publisher("notebook_traj", moveit_msgs.msg.DisplayTrajectory, queue_size=1)

In [ ]:
with open("/home/hri25-group3/ros/src/safety_perception/joint_snapshot1.pkl", "rb") as f:
        data = pickle.load(f)

In [ ]:
data[0]

{'t': 1761213852.0178509,
 'names': ['arm_shoulder_pan_joint',
  'arm_shoulder_lift_joint',
  'arm_elbow_joint',
  'arm_wrist_1_joint',
  'arm_wrist_2_joint',
  'arm_wrist_3_joint'],
 'q': [-3.1371305624591272,
  -2.5746733150877894,
  -1.473508358001709,
  -0.6813049477389832,
  1.5681782960891724,
  -0.3417566458331507]}

In [10]:
s = data[5]
print(s['names'])

['arm_shoulder_pan_joint', 'arm_shoulder_lift_joint', 'arm_elbow_joint', 'arm_wrist_1_joint', 'arm_wrist_2_joint', 'arm_wrist_3_joint']


In [12]:
ds = moveit_msgs.msg.DisplayRobotState()
ds.state.joint_state.name = s['names']
ds.state.joint_state.position = s['q']
state_pub.publish(ds)


In [ ]:
move = moveit_commander.MoveGroupCommander('arm')
move.set_max_velocity_scaling_factor(0.5)

move.set_named_target('ready')
move.go()


for i in range (len(data)-1):
    #start
    js1= JointState()
    js1.name=data[i]['names']
    js1.position=data[i]['q']
    move.set_joint_value_target(js1)
    move.set_max_velocity_scaling_factor(0.5)
    move.go()
    j=i+1
    for j in range (len(data)-1):
        #end
        js2= JointState()
        js2.name=data[i]['names']
        js2.position=data[i]['q']
        move.set_joint_value_target(js2)
        move.set_max_velocity_scaling_factor(0.5)
        move.go()
        #go back to start
        move.set_joint_value_target(js1)
        move.set_max_velocity_scaling_factor(0.5)
        move.go()
        
            


[INFO] [1762516782.273429746]: Loading robot model 'cu_ur5e'...
[INFO] [1762516782.276137063]: No root/virtual joint specified in SRDF. Assuming fixed joint
[INFO] [1762516784.522328499]: Ready to take commands for planning group arm.


[WARN] [1762516782.453788199]: Link operator_clearance has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.


MoveItCommanderException: Error setting joint target. Is the target within bounds?

In [13]:
with open("/home/hri25-group3/ros/src/safety_perception/good_pairs_fast.pkl", "rb") as f:
        plans = pickle.load(f)

In [14]:
plans[0,1]

joint_trajectory: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "base"
  joint_names: 
    - arm_shoulder_pan_joint
    - arm_shoulder_lift_joint
    - arm_elbow_joint
    - arm_wrist_1_joint
    - arm_wrist_2_joint
    - arm_wrist_3_joint
  points: 
    - 
      positions: [-3.1371927897082728, -2.5750514469542445, -1.4729422330856323, -0.6812046331218262, 1.568124771118164, -0.3418238798724573]
      velocities: [0.03255632944235696, 0.006502850424413667, 0.055635497965112704, -0.06283185307179588, 1.0580283829659567e-05, 3.1915709889456236e-06]
      accelerations: [3.2556329442356953, 0.6502850424413665, 5.5635497965112695, -6.283185307179587, 0.0010580283829659566, 0.0003191570988945623]
      effort: []
      time_from_start: 
        secs: 0
        nsecs:         0
    - 
      positions: [-3.120914624987094, -2.571800021742038, -1.4451244841030761, -0.7126205596577242, 1.5681300612600788, -0.34182228408696286]
      velocities: [0.325563

In [46]:
dtraj = moveit_msgs.msg.DisplayTrajectory()
rtraj = moveit_msgs.msg.RobotTrajectory()
rtraj.joint_trajectory = plans[0,5].joint_trajectory
dtraj.trajectory.append(rtraj)
traj_pub.publish(dtraj)

In [22]:
plans.keys()

dict_keys([(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 8), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 8), (2, 3), (2, 5), (2, 8), (4, 5), (4, 8)])

In [45]:
move = moveit_commander.MoveGroupCommander('arm')
move.set_max_velocity_scaling_factor(0.1)

start= data[0]

js1= JointState()
js1.name= start['names']
js1.position=start['q']



[INFO] [1762521315.612093266]: Ready to take commands for planning group arm.


In [53]:
move.set_joint_value_target(js1)# go to start 0 position

move.go()

True

In [49]:
k0 = [k for k in plans.keys() if isinstance(k, tuple) and len(k) >= 2 and k[0] == 0]
print(k0)




[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 8)]


In [81]:
if not k0:
    raise ValueError("No keys starting with 0 found.")

key = random.choice(k0)          # e.g., (0, 5)
plan = plans[key]                # access the plan
print("Picked key:", key)

endpoint=key[1]
#print(endpoint)

plans[key]


Picked key: (0, 2)


joint_trajectory: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "base"
  joint_names: 
    - arm_shoulder_pan_joint
    - arm_shoulder_lift_joint
    - arm_elbow_joint
    - arm_wrist_1_joint
    - arm_wrist_2_joint
    - arm_wrist_3_joint
  points: 
    - 
      positions: [-3.1374622027026575, -2.574627538720602, -1.4742820262908936, -0.680805043583252, 1.5682183504104614, -0.34173614183534795]
      velocities: [-0.036358630444698746, 0.006520455429575316, 0.055712901212019744, -0.06283185307179587, -8.992459303220712e-06, 5.201350043737559e-06]
      accelerations: [-3.635863044469874, 0.6520455429575315, 5.571290121201973, -6.283185307179585, -0.000899245930322071, 0.0005201350043737558]
      effort: []
      time_from_start: 
        secs: 0
        nsecs:         0
    - 
      positions: [-3.155641517925007, -2.5713673110058144, -1.4464255756848836, -0.71222097011915, 1.56821385418081, -0.34173354116032606]
      velocities: [-0.36358630

In [82]:
rtraj.joint_trajectory = plans[key].joint_trajectory


In [65]:
jt = rtraj.joint_trajectory
assert len(jt.points) > 0, "Chosen trajectory has no points."

# go to first waypoint so controllers don't abort
start_map = dict(zip(jt.joint_names, jt.points[0].positions))
move.set_joint_value_target(start_map)
ok = move.go(wait=True); move.stop()
print("Moved to start:", ok)

# execute the trajectory
ok = move.execute(rtraj, wait=True); move.stop()
print("Executed trajectory:", ok)


Moved to start: True
Executed trajectory: True


In [83]:
def has_outgoing(plans, node):
    # True if there exists any key (node, k) in plans
    return any(k[0] == node for k in plans)

In [84]:
def pick_edge_with_future(plans, i, avoid_edge=None):
    """Pick (i->j) preferring j that has outgoing edges. Avoid a specific edge if given."""
    outs = [k for k in plans if k[0] == i and k != avoid_edge]
    if not outs:
        return None
    # prefer edges whose destination isn't a dead-end
    good = [k for k in outs if has_outgoing(plans, k[1])]
    return random.choice(good if good else outs)

In [86]:
# choose from current index (e.g., cur_idx = 0)
key = pick_edge_with_future(plans, cur_idx)
if key is None:
    raise ValueError(f"No outgoing trajectories from {cur_idx}")

rtraj = plans[key]  # RobotTrajectory

# your existing execution snippet:
jt = rtraj.joint_trajectory
if not jt.points:
    raise ValueError("Chosen trajectory has no points.")

start_map = dict(zip(jt.joint_names, jt.points[0].positions))
move.set_joint_value_target(start_map)
ok = move.go(wait=True); move.stop()
print("Moved to start:", ok)

ok = move.execute(rtraj, wait=True); move.stop()
print("Executed trajectory:", ok)

# update to the new endpoint
cur_idx = key[1]
print("Now at index:", cur_idx)


ValueError: No outgoing trajectories from 8

In [75]:
def pick_key_with_backoff(plans, start_idx, max_steps=10):
    """
    Return a key (i,j) to run. If no outgoing from start_idx,
    try predecessors; if still none, pick any i with outgoing.
    """
    tried = set()
    i = start_idx
    for _ in range(max_steps):
        # 1) try outgoing from current i
        outs = [k for k in plans if k[0] == i]
        if outs:
            return random.choice(outs)

        tried.add(i)

        # 2) try stepping back to a predecessor of i
        preds = [k[0] for k in plans if k[1] == i and k[0] not in tried]
        if preds:
            i = random.choice(preds)
            continue

        # 3) fallback: any i with outgoing, not tried
        candidates = sorted({k[0] for k in plans if k[0] not in tried})
        if not candidates:
            return None
        i = random.choice(candidates)

    return None

cur_idx = key[1]   # or None

key = pick_key_with_backoff(plans, cur_idx)
if key is None:
    raise ValueError("No trajectories available anywhere.")

rtraj = plans[key]  # RobotTrajectory

jt = rtraj.joint_trajectory
if not jt.points:
    raise ValueError("Chosen trajectory has no points.")

# go to first waypoint so controllers don't abort
start_map = dict(zip(jt.joint_names, jt.points[0].positions))
move.set_joint_value_target(start_map)
ok = move.go(wait=True); move.stop()
print("Moved to start:", ok)

# execute
ok = move.execute(rtraj, wait=True); move.stop()
print("Executed trajectory:", ok)

# advance current index to the new endpoint
cur_idx = key[1]
print("Now at index:", cur_idx)


Moved to start: True
Executed trajectory: True
Now at index: 8
